In [44]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

c:\Users\Davay\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([2, 1])

 ### Parameter Access

In [16]:
net

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [15]:
net[0].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1524, -0.2269, -0.0838,  0.3527],
                      [-0.3396,  0.4917, -0.0110,  0.2439],
                      [-0.4935, -0.4033,  0.3169, -0.3528],
                      [-0.3169,  0.2204, -0.0646,  0.0563],
                      [ 0.4701,  0.2966, -0.4630, -0.3522],
                      [-0.1054,  0.1039,  0.4977,  0.1254],
                      [-0.1042, -0.3611, -0.4264, -0.1250],
                      [-0.2957,  0.1996,  0.0184,  0.4052]])),
             ('bias',
              tensor([ 0.2786,  0.0591,  0.3286,  0.4545,  0.4938,  0.1245, -0.2854, -0.4378]))])

In [20]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[-0.3355,  0.3066, -0.0034, -0.1305, -0.1553,  0.2115,  0.1366,  0.2557]])),
             ('bias', tensor([-0.3024]))])

### Targeted Parameters

Note that each parameter is represented as an instance of the parameter class. To do anything useful with the parameters, we first need to access the underlying numerical values. There are several ways to do this. Some are simpler while others are more general. The following code extracts the bias from the second neural network layer, which returns a parameter class instance, and further accesses that parameter’s value.

In [21]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.3024]))

Parameters are complex objects, containing values, gradients, and additional information. That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.

In [22]:
net[2].weight.grad == None

True

### Accessing All Parameters at Once

When we need to perform operations on all parameters, accessing them one-by-one can grow tedious. The situation can grow especially unwieldy when we work with more complex modules (e.g., nested modules), since we would need to recurse through the entire tree to extract each sub-module’s parameters. 
$\textbf{Below we demonstrate accessing the parameters of all layers.}$

In [33]:

[(name, param) for name, param in net.named_parameters()]

[('0.weight',
  Parameter containing:
  tensor([[ 0.1524, -0.2269, -0.0838,  0.3527],
          [-0.3396,  0.4917, -0.0110,  0.2439],
          [-0.4935, -0.4033,  0.3169, -0.3528],
          [-0.3169,  0.2204, -0.0646,  0.0563],
          [ 0.4701,  0.2966, -0.4630, -0.3522],
          [-0.1054,  0.1039,  0.4977,  0.1254],
          [-0.1042, -0.3611, -0.4264, -0.1250],
          [-0.2957,  0.1996,  0.0184,  0.4052]], requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([ 0.2786,  0.0591,  0.3286,  0.4545,  0.4938,  0.1245, -0.2854, -0.4378],
         requires_grad=True)),
 ('2.weight',
  Parameter containing:
  tensor([[-0.3355,  0.3066, -0.0034, -0.1305, -0.1553,  0.2115,  0.1366,  0.2557]],
         requires_grad=True)),
 ('2.bias',
  Parameter containing:
  tensor([-0.3024], requires_grad=True))]

### Tied Parameters

Often, we want to share parameters across multiple layers. Let’s see how to do this elegantly. In the following code we allocate a fully connected layer and then use its parameters specifically to set those of another layer. Here we need to run the forward propagation net(X) before accessing the parameters.

In [ ]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

### Exercises

1. Use the NestMLP model defined in Section 6.1 and access the parameters of the various layers.

In [40]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [46]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.0063, grad_fn=<SumBackward0>)

In [47]:
chimera[0].state_dict() # LazyLinear(64)

OrderedDict([('net.0.weight',
              tensor([[-0.0877, -0.3585, -0.2953, -0.1443],
                      [ 0.1216,  0.0731,  0.2099,  0.1792],
                      [ 0.1633,  0.4825,  0.4716, -0.4584],
                      [ 0.4360, -0.4739,  0.4699, -0.3693],
                      [-0.4357,  0.1256, -0.3125,  0.1569],
                      [-0.0351,  0.3847,  0.1077,  0.3922],
                      [ 0.2950, -0.3949, -0.2951, -0.1670],
                      [ 0.3111,  0.3471,  0.3015, -0.4196],
                      [ 0.2460, -0.1181,  0.0334, -0.3424],
                      [-0.4037, -0.3732,  0.4245, -0.0582],
                      [ 0.1236,  0.0804, -0.2163, -0.0618],
                      [ 0.0445,  0.0148,  0.3403, -0.3086],
                      [ 0.2415,  0.4747,  0.2716, -0.2019],
                      [ 0.3685, -0.3134, -0.3571,  0.1061],
                      [ 0.4501,  0.3378, -0.0096, -0.4528],
                      [-0.2607, -0.3124, -0.1625, -0.0790],
          

In [48]:
chimera[2].state_dict() # LazyLinear(32)

OrderedDict([('linear.weight',
              tensor([[-0.0042,  0.0870,  0.1771, -0.1444, -0.0761, -0.1452, -0.0505,  0.0137,
                        0.0983,  0.1258, -0.1773,  0.0215, -0.1587,  0.2031, -0.1518, -0.2098,
                        0.1247, -0.1080, -0.1618, -0.0983],
                      [-0.0761,  0.1452,  0.0302,  0.0623, -0.0286, -0.0878, -0.2207,  0.0261,
                        0.0422,  0.1852,  0.0320, -0.1799, -0.1590, -0.1376, -0.1658,  0.0840,
                        0.1087,  0.0202, -0.0120, -0.0416],
                      [ 0.0781, -0.0271,  0.1569,  0.2100, -0.1961, -0.1364,  0.0422,  0.1499,
                        0.1536,  0.2049,  0.0187, -0.0110,  0.2135, -0.0289,  0.1336,  0.2051,
                       -0.1148, -0.1143, -0.1050, -0.1406],
                      [-0.1714, -0.0527,  0.0056, -0.1244,  0.0910,  0.1127,  0.0014, -0.2120,
                       -0.1094,  0.1068,  0.0126,  0.1019,  0.2132, -0.0732,  0.0567, -0.0507,
                       -0.182

2. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.

3. Why is sharing parameters a good idea?